In [1]:
import datetime
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

ROOT_DIR = str(Path().resolve().parent)
sys.path.append(f"{ROOT_DIR}/src/")
plt.style.use(f'{ROOT_DIR}/notebooks/project.mplstyle')

# Running on Genomes

In [2]:
contig_filepath = f"{ROOT_DIR}/tests/test_data/GCA_000172155.1_ASM17215v1_genomic.fna.gz"
protein_filepath = f"{ROOT_DIR}/tests/test_data/GCA_000172155.1_ASM17215v1_protein.faa.gz"

## Prediction of growth conditions



In [3]:
from predict_physicochemistry import predict_physicochemistry

print(datetime.datetime.now())
predictions, genome_features = predict_physicochemistry(
        fna_path=contig_filepath,
        faa_path=protein_filepath,
        features_json=None,
        instructions_filename=f"{ROOT_DIR}/models/instructions.json",
        #save_intermediate=False,
    )
print(datetime.datetime.now())

pd.DataFrame(predictions).T.sort_index()

2023-11-29 15:20:27.600687
2023-11-29 15:20:34.650353


,value,lower_ci,upper_ci,warning
oxygen,0.981243,NaN,NaN,NaN
ph_max,8.728332,NaN,NaN,NaN
ph_min,5.345892,NaN,NaN,NaN
ph_optimum,7.243117,NaN,NaN,NaN
salinity_max,7.258816,NaN,NaN,NaN
salinity_min,2.509730,NaN,NaN,NaN
salinity_optimum,4.434118,NaN,NaN,NaN
temperature_max,35.297075,NaN,NaN,NaN
temperature_min,15.892330,NaN,NaN,NaN
temperature_optimum,27.934813,NaN,NaN,NaN


## Measure Genome Features

### 1 Protein Sequence

In [4]:
from protein import Protein

sequence = ''.join("""MAAQDVKQQTPYRVIQLEWDAEKGERNEAVGNFDELVTHHPKSNSDAHLVDGKVVGGQAG
RTLGVVGGEIQEIEVSKAGKDYGLRPDQVLLKKDFMLEDSRLPSGPSSRSLDVPSPVAGV
VGTVNTSKGLVDVLDREGGDVILRVRHMSPLHVKAGDQVEYGQALGVQGKQATGAIHVHM
EVDSRYYQHYENYVGDLVSGRLSIDAERRDRGIEPRPFVDDGTIRIGGSSEMVQKVQQTL
NAEGYRGADNQPLQEDGVYRLSMQAAVINYQQAHGLSQTGDIDPATLQQIAPRTFPPELN
REDHNATPTYRNLQGAVPSQDPLHRQAEEDVRRLEQSLGRDYDDNSARLAASSAHLAKAN
GLTQIDHVVLSNQTAAVGKGENVFVVQGALDNPAHLMAHMKTSDAIAQPVEQSLSQLQTL
SETQRQQQAQQQSQQQDQQQLSAPQHRMV""".split('\n'))

protein = Protein(sequence)
protein.protein_metrics()

{'pi': 5.382106971740723,
 'zc': 0.19821826280623608,
 'nh2o': 0.07137639198218272,
 'gravy': -0.6514476614699332,
 'thermostable_freq': 0.356347438752784,
 'length': 449,
 'is_exported': False,
 'aa_M': 0.0200445434298441,
 'aa_A': 0.08463251670378619,
 'aa_Q': 0.10690423162583519,
 'aa_D': 0.0757238307349666,
 'aa_V': 0.0935412026726058,
 'aa_K': 0.0334075723830735,
 'aa_T': 0.0400890868596882,
 'aa_P': 0.0467706013363029,
 'aa_Y': 0.026726057906458798,
 'aa_R': 0.062360801781737196,
 'aa_I': 0.031180400890868598,
 'aa_L': 0.08240534521158129,
 'aa_E': 0.05790645879732739,
 'aa_W': 0.0022271714922048997,
 'aa_G': 0.08685968819599109,
 'aa_N': 0.035634743875278395,
 'aa_F': 0.011135857461024499,
 'aa_H': 0.035634743875278395,
 'aa_S': 0.066815144766147}

### 1 Signal Peptide Prediction

In [5]:
from signal_peptide import SignalPeptideHMM

partial_sequence = 'MNKTLIAAAVAGIVLLASNAQAQTVPEGYQLQQVLMMSRHNLRAPLANNG'

signal_peptide_model = SignalPeptideHMM()
is_exported, signal_end_index = signal_peptide_model.predict_signal_peptide(partial_sequence)
signal_peptide = partial_sequence[:signal_end_index+1]
is_exported, signal_peptide

(True, 'MNKTLIAAAVAGIVLLASNAQA')

### 1 DNA Sequence


In [6]:
from dna import DNA

sequence = "GGATGGACGGAGGAATTCCTCAAGGAAGTCGGGCCCGCGCTGGTGGTACTCGGTCCAGGCTTCTTGCACGAAGAAGTCTCCGACCGCGCCTCTCTCCACCCTCCTGGCAAAATCGGCCAGTGACTTGATGCCGATGTGGTAGATGAAGCCGGTGTCGAGTACGCCCTTGGCGAAGGTCGGGTCCGCCAGCCTCTCGGGAAAATTTTCTTCGATGTTCAGATAGTAGCCGCGCATGCTGACGCCACGGCTATCTAAAGTGAACATTCCGTCCTTCAGAATGCAATCGGCATGGTGCCAAAATCCAGAGCCCACCGCGCTGGCAATCTTGTTGTCGTCAACTGCGTCGGATCGGACGAGGCACTTCGTGAACGGCGCCGGCAATTCGGACCGTCGAGTGTAATTACGGATTGCGGACCCAGACGGGCGCGTGGCGTGGCCGTTATGCCCAGCATTGACGGAGAACAGGTGGATTGCCCCCACTCCACCCGGAAAGCGTGAGATAAACTCTCGGATCGTGTCGTCGTTGCGCAGTAAAATAAATTCGTCAACATCGAGAAATATGTAATTACGGCTCAGATGTTTAAA"
dna = DNA(sequence)
dna.nucleotide_metrics()

{'nt_length': 585,
 'pur_pyr_transition_freq': 0.4897260273972603,
 'nt_C': 0.558974358974359,
 'nt_A': 0.441025641025641}

### All Protein and DNA Sequences

Note: It is expected and good to see NaN for many features because they cannot be computed by cellular localization.

In [ ]:
from genome import Genome
from signal_peptide import SignalPeptideHMM


genome_calc = Genome(
    contig_filepath=contig_filepath, 
    protein_filepath=protein_filepath,
)

print(datetime.datetime.now())
genome_features = genome_calc.genome_metrics()
print(datetime.datetime.now())

2023-11-29 15:20:34.677249


In [ ]:
genome_features.keys()

In [ ]:
df_genome = pd.DataFrame(genome_features)
df_genome.head(30)

### All Proteins

In [ ]:
protein_data = genome_calc.protein_data()
df_proteins = pd.DataFrame(protein_data).T
df_proteins

In [ ]:
x = 'gravy'

X = df_proteins[x].values 

fig, ax = plt.subplots()
ax.hist(X, bins=100)
#ax.axvline(0.5)

plt.show()

# Model Training

## Balancing a Dataset by Taxonomy

In [12]:
import sys
sys.path.append('../src/')
with open('../tests/test_data/test_genome_accessions.txt', 'r') as fh:
    genomes = [line.strip().split('.')[0] for line in fh.readlines()]

In [13]:
from taxonomy import TaxonomyGTDB

taxonomy = TaxonomyGTDB()
len(taxonomy.taxonomy_dict.keys())

402709

In [14]:
accession = list(taxonomy.taxonomy_dict.keys())[0]
{accession : taxonomy.taxonomy_dict['GCA_019399425']}

{'GCA_000979555': ('Bacteria',
  'Bacillota',
  'Bacilli',
  'Lactobacillales',
  'Streptococcaceae',
  'Streptococcus',
  'Streptococcus pneumoniae')}

In [15]:
#taxonomy.taxonomy_dict[]
list(taxonomy.taxonomy_dict.keys())[190000]

'GCA_017565965'

In [16]:
from taxonomy import BalanceTaxa

taxonomy = TaxonomyGTDB()
balancer = BalanceTaxa(taxonomy=taxonomy)
balanced_genomes = balancer.balance_dataset(
    genomes=genomes,
    proportion_to_keep=0.5,
    diversity_rank="species"
)
len(balanced_genomes) / len(genomes)

0.49996937588044343

## Partitioning (Splitting) a Dataset by Taxonomy

In [17]:
from taxonomy import PartitionTaxa

partitioner = PartitionTaxa(
    taxonomy=taxonomy,
    partition_rank='family',
    iteration_rank='phylum',
    diversity_rank='genus',
)

partitioned_genomes = partitioner.partition(balanced_genomes, partition_size=0.2)
nonpartitioned_genomes = set(balanced_genomes).difference(partitioned_genomes)
extended_partitioned_genomes = partitioner.find_relatives_of_partitioned_set_in_reference(partitioned_genomes)

print(len(partitioned_genomes) / len(balanced_genomes), len(nonpartitioned_genomes) / len(balanced_genomes))
print(len(partitioned_genomes), len(extended_partitioned_genomes))

0.20029400955531054 0.7997059904446895
1635 114071


In [18]:
# check for data leakage
partitioned_taxa = taxonomy.taxa_of_genomes(partitioned_genomes, partitioner.partition_rank)
nonpartitioned_taxa = taxonomy.taxa_of_genomes(nonpartitioned_genomes, partitioner.partition_rank)
assert len(partitioned_taxa.intersection(nonpartitioned_taxa)) == 0

# check partitioned genomes all present
len(set(partitioned_genomes).intersection(extended_partitioned_genomes)) == len(set(partitioned_genomes))
assert len(set(partitioned_genomes).difference(extended_partitioned_genomes)) == 0
print('Good!')

Good!


## Creating Cross-Validation Sets

In [ ]:
# waiting for model training kit

## Balancing a Data Numerically

In [ ]:
# waiting for model training kit

# Trait Data

## Download BacDive API Data

In [30]:
from model_training.download_training_data import QueryBacDive

credentials_filepath = f'../../.bacdive_credentials'

min_bacdive_id = 100
max_bacdive_id = 200
bacdive_dict = QueryBacDive(
            credentials_filepath=credentials_filepath,
            max_bacdive_id=int(max_bacdive_id),
            min_bacdive_id=int(min_bacdive_id),
        ).scrape_bacdive_api()

strain_ids = list(bacdive_dict.keys())
strain_data = bacdive_dict[strain_ids[0]]
strain_data.keys()

-- Authentication successful --


dict_keys(['General', 'Name and taxonomic classification', 'Morphology', 'Culture and growth conditions', 'Physiology and metabolism', 'Isolation, sampling and environmental information', 'Safety information', 'Sequence information', 'Genome-based predictions', 'External links', 'Reference'])

In [31]:
strain_data['General']

{'@ref': 3774,
 'BacDive-ID': 199,
 'DSM-Number': 10002,
 'keywords': ['genome sequence',
  '16S sequence',
  'Bacteria',
  'mesophilic',
  'animal pathogen'],
 'description': 'Arcanobacterium phocae M1590/94/3 is a mesophilic animal pathogen that was isolated from lung of common seal.',
 'NCBI tax id': {'NCBI tax id': 131112, 'Matching level': 'species'},
 'strain history': '<- G. Foster, M1590/94/3',
 'doi': '10.13145/bacdive199.20230509.8'}

## Parsing traits

In [29]:
from model_training.download_training_data import ComputeBacDiveTraits

strain_data = bacdive_dict[strain_ids[2]]
strain_traits = ComputeBacDiveTraits(strain_data).compute_trait_data()
strain_traits

{'ncbi_accession': 'GCA_900475915.1',
 'ncbi_taxid': 644284,
 'strain_id': 197,
 'species': 'Arcanobacterium haemolyticum',
 'ph_optimum': None,
 'ph_optimum_min': None,
 'ph_optimum_max': None,
 'temperature_optimum': 37.0,
 'salinity_optimum': None,
 'salinity_midpoint': None,
 'salinity_min': None,
 'salinity_max': None,
 'ph_min': None,
 'ph_max': None,
 'temperature_min': 37.0,
 'temperature_max': 37.0,
 'oxygen': 1,
 'use_ph_optimum': False,
 'use_temperature_optimum': False,
 'use_salinity_optimum': False,
 'use_ph_range': False,
 'use_temperature_range': False,
 'use_salinity_range': False,
 'use_oxygen': True,
 'aerobe': None,
 'anaerobe': 1,
 'microaerophile': None,
 'facultative anaerobe': 1,
 'obligate aerobe': None,
 'obligate anaerobe': None,
 'facultative aerobe': None,
 'aerotolerant': None,
 'microaerotolerant': None}